# importing packages

In [1]:
import pandas as pd
import numpy as np
import os

# directory for getting the original feature list

In [2]:
#find current directory
#location of original data
starting_directory = os.getcwd()

new_dirrectory = "C:/Users/maega/Documents/3000 PhD/3300_BEAT Labs/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/012 Processed Data/PtLabsFiltered_2024-06-26_v1"
os.chdir(new_dirrectory)
#change working directory to where the data is stored

#create file naming system for data outputs

### importing the data

In [3]:
#import the original feature list
lab_list_import = pd.read_excel("ListPts-W-WO-Lab-20240626_v2.xlsx", sheet_name = "lab list pts without", header = [0])


### collecting the lab names and the number of patients as seperate series

In [4]:
lab_list = lab_list_import.iloc[:,0] #list of the raw lab result names without transformations
lab_list_numbpts = lab_list_import.iloc[:,2] #for use later in assessing the completeness of the data

### additional features describing patients which arean't labs in their untransformed states
no descriptive statistics or fits

In [5]:
# #additional starting features
# additional_features = pd.Series(["Amyloid Status", "Dx", "Sex", 'Age', 'Survival (in months)',
#                        "ChemoTx", "ChemoTx Number", "ImmunoTx", "ImmunoTx Number", "HemeTx", "HemeTx Number", 
#                        "RadTx", "RadTx Number", "OtherTx", "OtherTx Number", 'time from diagnosis to note (months)', 
#                        'ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease', 
#                        "1st_EF(avg%)", "1st_Diastolic grade", "1st_BSA", "1st_PWT(cm)", "1st_IVS(cm)", "1st_lvid(cm)", 
#                        "1st_LA_dimension (cm)","1st_Lvmass", "1st_LVMI","1st_RWT", "1st_LA_grade", "1st_mv_grade", 
#                        "1st_AV_grade", "1st_rvsp(mmHg)", "1st_LVH", "1stGLS", "1stGLSreported", "use1stGLS", 
#                        "1st_outerLS", "1st_midLS", "1st_innerLS", "1st_echoOSH"])

In [21]:
#additional starting features
additional_features = pd.Series(["Amyloid Status", "Dx", "Sex", 'Age', 'Survival (in months)',
                       "ChemoTx", "ImmunoTx", "HemeTx", 
                       "RadTx", "OtherTx", 'time from diagnosis to note (months)', 
                       'ECOG Performance Status', 'Karnofsky Performance Status', 'Clinical Impression of Disease', 
                       "1st_EF(avg%)", "1st_Diastolic grade", "1st_BSA", "1st_PWT(cm)", "1st_IVS(cm)", "1st_lvid(cm)", 
                       "1st_LA_dimension (cm)","1st_Lvmass", "1st_LVMI","1st_RWT", "1st_LA_grade", "1st_mv_grade", 
                       "1st_AV_grade", "1st_rvsp(mmHg)", "1st_LVH", "1stGLS", "1stGLSreported", "use1stGLS", 
                       "1st_outerLS", "1st_midLS", "1st_innerLS", "1st_echoOSH", "time from diagnosis to first echo (months)"])
#here the "ChemoTx Number" and similar original data were removed as they will be picked up as transformations

In [22]:
full_og_featureList = pd.concat([lab_list, additional_features], axis = 0) #adding the two lists of features together
full_og_featureList.reset_index()

,index,0
0,0,Creatinine Result(mg/dL)
1,1,Potassium Result(mmol/L)
2,2,total CO2 Result
3,3,Chloride Result(mmol/L)
4,4,Calcium Result(mg/dL)
...,...,...
188,32,1st_outerLS
189,33,1st_midLS
190,34,1st_innerLS
191,35,1st_echoOSH


In [23]:
#get the list of best features from the SVM

# get the list of best features from the SVM models 

In [24]:
#find current directory
#location of original data
starting_directory = os.getcwd()

new_dirrectory = 'C:/Users/maega/UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/013 Analyzed Data outputs'
# "C:/Users/mcremer/UFL Dropbox/Maegan Cremer/research-share/Maegan/Projects/Cardiac-Amyloidosis-Multiple-Myeloma/013 Analyzed Data outputs"
#C:/Users/maega/UFL Dropbox/Maegan Cremer/research-share/Maegan
os.chdir(new_dirrectory)
#change working directory to where the data is stored


### importing the list of best features by model and finding the unique list of features + transformations

In [25]:
#importing the dataframe describing features selected for each model
features_across_models = pd.read_excel("2024-07-29_FeatureListFromSVM.xlsx", sheet_name = "feature list", header = [0,1,2])
# features_across_top38 = pd.read_excel("2024-07-29_FeatureListFromSVM.xlsx", sheet_name = "feature list top 38", header = [0,1,2])
# features_across_allLabs = pd.read_excel("2024-07-29_FeatureListFromSVM.xlsx", sheet_name = "feature list all labs", header = [0,1,2])

#create 1D list of features across all models
features_1D = features_across_models.to_numpy().flatten()

features_1D_clean = features_1D[~pd.isna(features_1D)]
#create list of unique features across all models
features_1D_unique = pd.unique(features_1D_clean)

### Describing the frequency of features across all models

In [26]:
#characterize the frequency of the transformed feature in the list of features selected by SVM
from collections import Counter
feature_counts = Counter(features_1D_clean)
feature_counts_dict = dict(feature_counts)


# Comparing the original data types to the list of features
makes a dictionary where the keys are the original data types represented among the features\
values are the best features represented across all models

In [27]:
#find the original list of labs within the transformed list
#filters the unique list of labs

dict_og_to_transformed = {}
dict_og_to_trimmed_transformed = {} #for use in multi-indexing
# list_transformedFeatures = []
for og_feature in full_og_featureList:
    list_transformedFeatures = []
    list_transformations = []
    for feature_transformed in features_1D_unique:
        if og_feature in feature_transformed:
            # list_transformedFeatures.append(feature_transformed)
            #trimming slicing the original feature off and adding the transformed section to a list
            start_position_for_slice = len(og_feature) + 1 #removes the space when slicing
            name_transformation = feature_transformed[start_position_for_slice:] #slices the length of the orginal feature off the string
            #check to see if the section trimmed from the feature exactly matches the og_feature
            trimmed_head_string = feature_transformed[:len(og_feature)]
            if trimmed_head_string == og_feature: #ensures that the feature identified isn't a variation such as PCT myelocyte vs myelocyte
                if name_transformation != "eported": #a little weirdness from one of the echo measures
                    list_transformations.append(name_transformation) #adds the transformation to the list for that feature
                    list_transformedFeatures.append(feature_transformed) #adds the og+transform to the og_to_transform
    if len(list_transformedFeatures) != 0:
        dict_og_to_transformed[og_feature] = list_transformedFeatures
        dict_og_to_trimmed_transformed[og_feature] = list_transformations

In [28]:
#creating multi-indexed pattern for the table of features used in the SVM models 

#collecting the two arrays for indexing from the dictionary
level_1 = [] #keys
level_2 = [] #values, or the transformations

#to make the key list the same length as the value list such that the pairings can be made... 
for key in list(dict_og_to_trimmed_transformed.keys()):
    for transformation in list(dict_og_to_trimmed_transformed[key]):
        level_1.append(key)
        level_2.append(transformation)

level_1_2 = [level_1, level_2]
multiIndex_tuple_features = list(zip(*level_1_2))

### create binary dataframe of features per model with multiIndex

In [29]:
#set the unique features to the row index 
row_dictionary = dict_og_to_transformed

#set the columns to the headers from earlier, the models
column_index = features_across_models.columns

dict_binary = {}
#pull the features from a column and compare them to 
for column in column_index: 
    column_data_binary = []
    column_data = features_across_models[column]
    for key in list(row_dictionary.keys()):
        for feature in list(row_dictionary[key]):
            if feature in column_data.values:
                column_data_binary.extend([1])
            else:
                column_data_binary.extend([0])
    dict_binary[column] = column_data_binary       



multiIndex_features = pd.MultiIndex.from_tuples(multiIndex_tuple_features, names=["original data", "transformation"])
df_binary = pd.DataFrame.from_dict(dict_binary, orient = "columns")

df_binary.index = multiIndex_features

# organizing the frequency information to a multiIndexed dataframe


In [30]:
#set the unique features to the row index 
row_dictionary = dict_og_to_transformed

dict_frequencyList = {}
for key in list(row_dictionary.keys()):
    for feature in list(row_dictionary[key]):
        dict_frequencyList[feature] = feature_counts_dict[feature] #reorders the dictionary?


multiIndex_features = pd.MultiIndex.from_tuples(multiIndex_tuple_features, names=["original data", "transformation"])
df_featureFrequency = pd.DataFrame.from_dict(dict_frequencyList, orient = "index")

df_featureFrequency.index = multiIndex_features 

In [31]:
# #figure out where the duplicates are

# #  = [' '.join(map(str, col)).strip() for col in dict_og_to_trimmed_transformed]
# test_compress_1 =[group + ' ' + transform for group in list(dict_og_to_trimmed_transformed.keys()) 
#                   for transform in list(dict_og_to_trimmed_transformed[group])]



# Illustrating data completeness for features selected by a model

copy the binary dataframe.  Then replace row column pairs which are 1 to the number of patients with that original piece of data